In [1]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [2]:
os.system('gcloud compute instances delete --zone=us-central1-a --quiet --project=ucr-ursa-major-lesani-lab $(gcloud compute instances list --project=ucr-ursa-major-lesani-lab --filter="name~^tmane-" --format="value(name)")')

ERROR: (gcloud.compute.instances.delete) argument INSTANCE_NAMES [INSTANCE_NAMES ...]: Must be specified.
Usage: gcloud compute instances delete INSTANCE_NAMES [INSTANCE_NAMES ...] [optional flags]
  optional flags may be  --delete-disks | --help | --keep-disks | --zone

For detailed information on this command and its flags, run:
  gcloud compute instances delete --help


512

In [3]:
num_nodes = 32

fialed_node_id = 9


operationrate =1000000000

In [4]:
gcp_username = 'tejas'

In [5]:

commands  = []



for i in range(num_nodes):
    cmd =  'gcloud compute instances create tmane-'+f"{i:03}"+' \
    --project=ucr-ursa-major-lesani-lab \
    --zone=us-central1-a \
    --machine-type=e2-highcpu-8 \
    --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
    --maintenance-policy=MIGRATE \
    --provisioning-model=STANDARD \
    --service-account=961693926925-compute@developer.gserviceaccount.com \
    --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/trace.append \
    --create-disk=auto-delete=yes,boot=yes,device-name=instance-20241002-130909,image=projects/rhel-cloud/global/images/rhel-8-v20240910,mode=rw,size=20,type=pd-balanced \
    --no-shielded-secure-boot \
    --shielded-vtpm \
    --shielded-integrity-monitoring \
    --labels=goog-ec-src=vm_add-gcloud \
    --reservation-affinity=any'

    commands.append(cmd)


def run_command(command):
    subprocess.call(command, shell=True)


# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-000].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-007].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.

 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-006].


NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
tmane-000  us-central1-a  e2-highcpu-8               10.128.0.13  136.114.95.227  RUNNING
NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
tmane-007  us-central1-a  e2-highcpu-8               10.128.0.18  34.173.155.252  RUNNING


 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
tmane-006  us-central1-a  e2-highcpu-8               10.128.0.17  34.172.113.32  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-028].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



...

NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
tmane-027  us-central1-a  e2-highcpu-8               10.128.0.20  35.232.50.38  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-013].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
tmane-013  us-central1-a  e2-highcpu-8               10.128.0.30  34.42.157.63  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-019].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP   EXTERNAL_IP   STATUS
tmane-019  us-central1-a  e2-highcpu-8               10.128.0.109  34.10.239.49  RUNNING


In [6]:
time.sleep(60)

In [7]:
os.system('git add .; git commit -m "test"; git push -f origin HEAD:main')


[main 2b55739] test
 1 file changed, 221 insertions(+), 18033 deletions(-)


To github.com:tejas-shivanand-mane/Optimistic-Replication.git
   dce03fd..2b55739  HEAD -> main


0

In [8]:
def clean_node(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd; sudo rm -r *"'
    )
    output = subprocess.getoutput(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(clean_node)(i) for i in range(num_nodes))

rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*

In [9]:
def git_clone(i):

    command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"'

    print(command)

    output = os.system(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(git_clone)(i) for i in range(num_nodes))


gcloud compute ssh --zone "us-central1-a" "tmane-030" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-004" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-002" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-003" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Re

Operation aborted.


Dependencies resolved.
 Package                         Arch    Version                                      Repository                                   Size
Installing:
 kernel                          x86_64  4.18.0-553.105.1.el8_10                      rhui-rhel-8-for-x86_64-baseos-rhui-rpms      11 M
Upgrading:
 NetworkManager                  x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     2.3 M
 NetworkManager-libnm            x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     1.9 M
 NetworkManager-team             x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     160 k
 NetworkManager-tui              x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     356 k
 audit                           x86_64  3.1.2-1.el8_10.1                             rhui-rhel-8-for-x86_64-baseos-rhui-rpms     266 k
 a

Operation aborted.
Operation aborted.


Last metadata expiration check: 0:00:16 ago on Thu 19 Feb 2026 09:07:58 PM UTC.
Dependencies resolved.
 Package                         Arch    Version                                      Repository                                   Size
Installing:
 kernel                          x86_64  4.18.0-553.105.1.el8_10                      rhui-rhel-8-for-x86_64-baseos-rhui-rpms      11 M
Upgrading:
 NetworkManager                  x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     2.3 M
 NetworkManager-libnm            x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     1.9 M
 NetworkManager-team             x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     160 k
 NetworkManager-tui              x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     356 k
 audit                           x86_64  3.1.2-1.el8_10.1 

...

0
Dependencies resolved.
 Package                         Arch    Version                                      Repository                                   Size
Installing:
 kernel                          x86_64  4.18.0-553.105.1.el8_10                      rhui-rhel-8-for-x86_64-baseos-rhui-rpms      11 M
Upgrading:
 NetworkManager                  x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     2.3 M
 NetworkManager-libnm            x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     1.9 M
 NetworkManager-team             x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     160 k
 NetworkManager-tui              x86_64  1:1.40.16-20.el8_10                          rhui-rhel-8-for-x86_64-baseos-rhui-rpms     356 k
 audit                           x86_64  3.1.2-1.el8_10.1                             rhui-rhel-8-for-x86_64-baseos-rhui-rpms     266 k


Operation aborted.


Last metadata expiration check: 0:00:27 ago on Thu 19 Feb 2026 09:09:23 PM UTC.
Dependencies resolved.
 Package                  Arch    Version                                 Repository                                   Size
Installing:
 git                      x86_64  2.43.7-1.el8_10                         rhui-rhel-8-for-x86_64-appstream-rhui-rpms   93 k
Installing dependencies:
 emacs-filesystem         noarch  1:26.1-15.el8_10                        rhui-rhel-8-for-x86_64-baseos-rhui-rpms      71 k
 git-core                 x86_64  2.43.7-1.el8_10                         rhui-rhel-8-for-x86_64-appstream-rhui-rpms   11 M
 git-core-doc             noarch  2.43.7-1.el8_10                         rhui-rhel-8-for-x86_64-appstream-rhui-rpms  3.1 M
 perl-Carp                noarch  1.42-396.el8                            rhui-rhel-8-for-x86_64-baseos-rhui-rpms      30 k
 perl-Data-Dumper         x86_64  2.167-399.el8                           rhui-rhel-8-for-x86_64-baseos-rhui-rpms   

Cloning into 'Optimistic-Replication'...


0


In [10]:
def kill_code(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "sudo killall main-replication-protocols"'
    )
    output = subprocess.getoutput(command)
    print(output)


In [11]:

num_txns = 12000
write_percent = 100

usecase = 'ycsb'


## RESTART

In [12]:
# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))

    
# os.system('git add .; git commit -m "test"; git push -f origin HEAD:main')

# def git_pull(i):

#     command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"'

#     print(command)

#     output = os.system(command)
#     print(output)
# results = Parallel(n_jobs=60, prefer="threads")(delayed(git_pull)(i) for i in range(num_nodes))

# def setup(i):
#     command = (
#         f'gcloud compute ssh --zone us-central1-a '
#         f'{gcp_username}@tmane-{i:03} '
#         f'--project ucr-ursa-major-lesani-lab '
#         f'--command "cd Optimistic-Replication; cp optrep_script.sh ../optrep_script.sh; cd ../; sh optrep_script.sh {num_nodes} {num_txns} {write_percent} {usecase} > /home/{gcp_username}/log.txt"'
#     )
#     os.system(command)

    

# results = Parallel(n_jobs=60, prefer="threads")(delayed(setup)(i) for i in range(num_nodes))

# result = subprocess.check_output('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)"', shell=True, text=True)

# # Convert the output to a list by splitting on newlines
# ip_list = result.splitlines()

# ip_list

# def get_sorted_instance_ips():
#     result = subprocess.check_output(
#         "gcloud compute instances list --filter='name~^tmane-' "
#         "--format='value(name,networkInterfaces[0].networkIP)'",
#         shell=True
#     )
#     lines = result.decode().splitlines()
#     # Parse into list of tuples: (name, ip)
#     name_ip_pairs = [line.strip().split() for line in lines]
#     # Sort by name: optrep-000, optrep-001, ...
#     sorted_pairs = sorted(name_ip_pairs, key=lambda x: x[0])
#     # Extract sorted IPs
#     sorted_ips = [ip for name, ip in sorted_pairs]
#     return sorted_ips

# ip_list = get_sorted_instance_ips()
# ip_string = " ".join(ip_list)

# ip_string

# def run_code(i):
#     output_dir = f"../../wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}"
#     output_file = f"{output_dir}/result{i}.out"
#     command = (
#         f'gcloud compute ssh --zone us-central1-a '
#         f'{gcp_username}@tmane-{i:03} '
#         f'--project ucr-ursa-major-lesani-lab '
#         f'--command "cd Optimistic-Replication/tcp/build; '
#         f'sudo bash -c \''
#         f'mkdir -p {output_dir}; '
#         f'nohup ./main-replication-protocols {i+1} {num_nodes} {num_txns} {write_percent} {usecase} {operationrate} {fialed_node_id} {ip_string} > {output_file} 2>&1 &'
#         f'\'"'
#     )
#     print(command)
#     output = subprocess.getoutput(command)
#     print(output)



# 1 failure

In [13]:
# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
# time.sleep(7)

# for i in range(num_nodes):
#     run_code(i)


# time.sleep(35)  # Let it run for 30 seconds
# kill_code(7)   
# time.sleep(17)
# # kill_code(6)   
# time.sleep(17)
# # kill_code(5)   

# time.sleep(7)


# time.sleep(65) 


# # time.sleep(25)  # Let it run for 30 seconds
# # kill_code(3)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(6)
# # # kill_code(2)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(54) 


# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))



# if not os.path.exists('../optrep_outputs'):
#     os.mkdir('../optrep_outputs')

# for i in range(num_nodes):
#     command = (
#         f'gcloud compute scp --zone us-central1-a '
#         f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}/result{i}.out '
#         f'../optrep_outputs/{i}.txt '
#         f'--project ucr-ursa-major-lesani-lab'
#     )
#     os.system(command)
#     print('copied')

#     # break

# src_dir = '../optrep_outputs/'
# dst_dir = f'../OptimisticReplicationOuts/{usecase}_1'

# # Make sure destination folder exists
# os.makedirs(dst_dir, exist_ok=True)

# # Iterate through all files in the source directory
# for filename in os.listdir(src_dir):
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)

#     # Copy only files (skip directories)
#     if os.path.isfile(src_path):
#         shutil.copy2(src_path, dst_path)

# 3 failure

In [14]:
# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
# time.sleep(5)

# for i in range(num_nodes):
#     run_code(i)


# time.sleep(35)  # Let it run for 30 seconds
# kill_code(7)   
# time.sleep(17)
# kill_code(6)   
# time.sleep(17)
# kill_code(5)   

# time.sleep(7)


# time.sleep(65) 


# # time.sleep(25)  # Let it run for 30 seconds
# # kill_code(3)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(6)
# # # kill_code(2)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(54) 


# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))



# if not os.path.exists('../optrep_outputs'):
#     os.mkdir('../optrep_outputs')

# for i in range(num_nodes):
#     command = (
#         f'gcloud compute scp --zone us-central1-a '
#         f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}/result{i}.out '
#         f'../optrep_outputs/{i}.txt '
#         f'--project ucr-ursa-major-lesani-lab'
#     )
#     os.system(command)
#     print('copied')

#     # break

# src_dir = '../optrep_outputs/'
# dst_dir = f'../OptimisticReplicationOuts/{usecase}_3'

# # Make sure destination folder exists
# os.makedirs(dst_dir, exist_ok=True)

# # Iterate through all files in the source directory
# for filename in os.listdir(src_dir):
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)

#     # Copy only files (skip directories)
#     if os.path.isfile(src_path):
#         shutil.copy2(src_path, dst_path)

# 2 failure

In [15]:
# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
# time.sleep(5)

# for i in range(num_nodes):
#     run_code(i)


# time.sleep(35)  # Let it run for 30 seconds
# kill_code(7)   
# time.sleep(17)
# kill_code(6)   
# time.sleep(17)
# # kill_code(5)   

# time.sleep(7)


# time.sleep(65) 

# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))

# if not os.path.exists('../optrep_outputs'):
#     os.mkdir('../optrep_outputs')

# for i in range(num_nodes):
#     command = (
#         f'gcloud compute scp --zone us-central1-a '
#         f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}/result{i}.out '
#         f'../optrep_outputs/{i}.txt '
#         f'--project ucr-ursa-major-lesani-lab'
#     )
#     os.system(command)
#     print('copied')

#     # break

# src_dir = '../optrep_outputs/'
# dst_dir = f'../OptimisticReplicationOuts/{usecase}_2'

# # Make sure destination folder exists
# os.makedirs(dst_dir, exist_ok=True)

# # Iterate through all files in the source directory
# for filename in os.listdir(src_dir):
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)

#     # Copy only files (skip directories)
#     if os.path.isfile(src_path):
#         shutil.copy2(src_path, dst_path)



# 0 failure

In [16]:
# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
# time.sleep(5)

# for i in range(num_nodes):
#     run_code(i)


# time.sleep(35)  # Let it run for 30 seconds
# # kill_code(7)   
# time.sleep(17)
# # kill_code(6)   
# time.sleep(17)
# # kill_code(5)   

# time.sleep(7)


# time.sleep(65) 


# # time.sleep(25)  # Let it run for 30 seconds
# # kill_code(3)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(6)
# # # kill_code(2)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(54) 


# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))



# if not os.path.exists('../optrep_outputs'):
#     os.mkdir('../optrep_outputs')

# for i in range(num_nodes):
#     command = (
#         f'gcloud compute scp --zone us-central1-a '
#         f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}/result{i}.out '
#         f'../optrep_outputs/{i}.txt '
#         f'--project ucr-ursa-major-lesani-lab'
#     )
#     os.system(command)
#     print('copied')

#     # break

# src_dir = '../optrep_outputs/'
# dst_dir = f'../OptimisticReplicationOuts/{usecase}_0'

# # Make sure destination folder exists
# os.makedirs(dst_dir, exist_ok=True)

# # Iterate through all files in the source directory
# for filename in os.listdir(src_dir):
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)

#     # Copy only files (skip directories)
#     if os.path.isfile(src_path):
#         shutil.copy2(src_path, dst_path)

## Normal Run


In [17]:
# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
# time.sleep(5)

# for i in range(num_nodes):
#     run_code(i)


# time.sleep(150)  # Let it run for 30 seconds
# # kill_code(7)   
# # time.sleep(17)
# # kill_code(6)   
# # time.sleep(17)
# # kill_code(5)   

# # time.sleep(7)


# # time.sleep(65) 


# # time.sleep(25)  # Let it run for 30 seconds
# # kill_code(3)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(6)
# # # kill_code(2)    # Kill node 3 (id=3, 0-indexed position 2)
# # time.sleep(54) 


# results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))



# if not os.path.exists('../optrep_outputs'):
#     os.mkdir('../optrep_outputs')

# for i in range(num_nodes):
#     command = (
#         f'gcloud compute scp --zone us-central1-a '
#         f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}/result{i}.out '
#         f'../optrep_outputs/{i}.txt '
#         f'--project ucr-ursa-major-lesani-lab'
#     )
#     os.system(command)
#     print('copied')

#     # break

# src_dir = '../optrep_outputs/'
# dst_dir = f'../OptimisticReplicationOuts/{usecase}_num_nodes_{num_nodes}_nr_writepercent_{write_percent}'

# # Make sure destination folder exists
# os.makedirs(dst_dir, exist_ok=True)

# # Iterate through all files in the source directory
# for filename in os.listdir(src_dir):
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)

#     # Copy only files (skip directories)
#     if os.path.isfile(src_path):
#         shutil.copy2(src_path, dst_path)

In [18]:

# src_dir = '../optrep_outputs/'
# dst_dir = '../OptimisticReplicationOuts/course_0'

# # Make sure destination folder exists
# os.makedirs(dst_dir, exist_ok=True)

# # Iterate through all files in the source directory
# for filename in os.listdir(src_dir):
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)

#     # Copy only files (skip directories)
#     if os.path.isfile(src_path):
#         shutil.copy2(src_path, dst_path)

In [19]:
# os.system('gcloud compute instances delete --zone=us-central1-a --quiet --project=ucr-ursa-major-lesani-lab $(gcloud compute instances list --project=ucr-ursa-major-lesani-lab --filter="name~^tmane-" --format="value(name)")')

In [ ]:
for write_percent in [10, 20]:

    num_txns = 12000
    usecase = 'set'
    
    results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
    
        
    os.system('git add .; git commit -m "working"; git push -f origin HEAD:main')
    
    def git_pull(i):
    
        command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"'
    
        print(command)
    
        output = os.system(command)
        print(output)
    results = Parallel(n_jobs=60, prefer="threads")(delayed(git_pull)(i) for i in range(num_nodes))
    
    def setup(i):
        command = (
            f'gcloud compute ssh --zone us-central1-a '
            f'{gcp_username}@tmane-{i:03} '
            f'--project ucr-ursa-major-lesani-lab '
            f'--command "cd Optimistic-Replication; cp optrep_script.sh ../optrep_script.sh; cd ../; sh optrep_script.sh {num_nodes} {num_txns} {write_percent} {usecase} > /home/{gcp_username}/log.txt"'
        )
        os.system(command)
    
        
    
    results = Parallel(n_jobs=60, prefer="threads")(delayed(setup)(i) for i in range(num_nodes))
    
    result = subprocess.check_output('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)"', shell=True, text=True)
    
    # Convert the output to a list by splitting on newlines
    ip_list = result.splitlines()
    
    ip_list
    
    def get_sorted_instance_ips():
        result = subprocess.check_output(
            "gcloud compute instances list --filter='name~^tmane-' "
            "--format='value(name,networkInterfaces[0].networkIP)'",
            shell=True
        )
        lines = result.decode().splitlines()
        # Parse into list of tuples: (name, ip)
        name_ip_pairs = [line.strip().split() for line in lines]
        # Sort by name: optrep-000, optrep-001, ...
        sorted_pairs = sorted(name_ip_pairs, key=lambda x: x[0])
        # Extract sorted IPs
        sorted_ips = [ip for name, ip in sorted_pairs]
        return sorted_ips
    
    ip_list = get_sorted_instance_ips()
    ip_string = " ".join(ip_list)
    
    ip_string
    
    def run_code(i):
        output_dir = f"../../wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}"
        output_file = f"{output_dir}/result{i}.out"
        command = (
            f'gcloud compute ssh --zone us-central1-a '
            f'{gcp_username}@tmane-{i:03} '
            f'--project ucr-ursa-major-lesani-lab '
            f'--command "cd Optimistic-Replication/tcp/build; '
            f'sudo bash -c \''
            f'mkdir -p {output_dir}; '
            f'nohup ./main-replication-protocols {i+1} {num_nodes} {num_txns} {write_percent} {usecase} {operationrate} {fialed_node_id} {ip_string} > {output_file} 2>&1 &'
            f'\'"'
        )
        print(command)
        output = subprocess.getoutput(command)
        print(output)
    
    
    
    
    results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
    time.sleep(5)
    

    for i in range(num_nodes):
        run_code(i)
    
    
    time.sleep(160)  # Let it run for 30 seconds
    # kill_code(7)   
    # time.sleep(17)
    # kill_code(6)   
    # time.sleep(17)
    # kill_code(5)   
    
    # time.sleep(7)
    
    
    # time.sleep(65) 
    
    
    # time.sleep(25)  # Let it run for 30 seconds
    # kill_code(3)    # Kill node 3 (id=3, 0-indexed position 2)
    # time.sleep(6)
    # # kill_code(2)    # Kill node 3 (id=3, 0-indexed position 2)
    # time.sleep(54) 
    
    
    results = Parallel(n_jobs=10, prefer="threads")(delayed(kill_code)(i) for i in range(num_nodes))
    
    
        
    path = '../optrep_outputs'
    
    if os.path.exists(path):
        shutil.rmtree(path)
    
    os.mkdir(path)
        
    for i in range(num_nodes):
        command = (
            f'gcloud compute scp --zone us-central1-a '
            f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}/result{i}.out '
            f'../optrep_outputs/{i}.txt '
            f'--project ucr-ursa-major-lesani-lab'
        )
        os.system(command)
        print('copied')
        if i>3:
            
            break
    
    src_dir = '../optrep_outputs/'
    dst_dir = f'../OptimisticReplicationOuts/test{usecase}_num_nodes_{num_nodes}_nr_writepercent_{write_percent}'
    
    # Make sure destination folder exists
    os.makedirs(dst_dir, exist_ok=True)
    
    # Iterate through all files in the source directory
    for filename in os.listdir(src_dir):
        src_path = os.path.join(src_dir, filename)
        dst_path = os.path.join(dst_dir, filename)
    
        # Copy only files (skip directories)
        if os.path.isfile(src_path):
            shutil.copy2(src_path, dst_path)

main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-repli

To github.com:tejas-shivanand-mane/Optimistic-Replication.git
   050a471..85f37e1  HEAD -> main


gcloud compute ssh --zone "us-central1-a" "tmane-030" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-002" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-000" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-005" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-007" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-004" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-006" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud

From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main


Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
Updating 050a471..85f37e1
Fast-forward
Updating 050a471..85f37e1
Fast-forward
Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168

...

0
Updating 050a471..85f37e1
Fast-forward
Updating 050a471..85f37e1
Fast-forward
Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
Updating 050a471..85f37e1
Fast-forward
Updating 050a471..85f37e1
Fast-forward
Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++

From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..85f37e1  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   050a471..

Updating 050a471..85f37e1
Fast-forward
 OptimisticReplication.ipynb | 4263 ++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 4168 insertions(+), 95 deletions(-)
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In file included from set-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
mkdir: cannot create directory ‘workload’: File exists
In file included from set-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
In file included from set-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
In file included from set-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
In file included from set-benchmark.cpp:12:
project.hpp: In member fu

main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-repli

In [21]:

        
    # path = '../optrep_outputs'
    
    # if os.path.exists(path):
    #     shutil.rmtree(path)
    
    # os.mkdir(path)
        
    # for i in range(num_nodes):
    #     command = (
    #         f'gcloud compute scp --zone us-central1-a '
    #         f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}/result{i}.out '
    #         f'../optrep_outputs/{i}.txt '
    #         f'--project ucr-ursa-major-lesani-lab'
    #     )
    #     os.system(command)
    #     print('copied')
    #     if i>5:
            
    #         break
    
    # src_dir = '../optrep_outputs/'
    # dst_dir = f'../OptimisticReplicationOuts/test_{usecase}_num_nodes_{num_nodes}_nr_writepercent_{write_percent}'
    
    # # Make sure destination folder exists
    # os.makedirs(dst_dir, exist_ok=True)
    
    # # Iterate through all files in the source directory
    # for filename in os.listdir(src_dir):
    #     src_path = os.path.join(src_dir, filename)
    #     dst_path = os.path.join(dst_dir, filename)
    
    #     # Copy only files (skip directories)
    #     if os.path.isfile(src_path):
    #         shutil.copy2(src_path, dst_path)

In [22]:
dst_dir

'../OptimisticReplicationOuts/testset_num_nodes_32_nr_writepercent_15'